In [ ]:
import sys
sys.path.insert(0, '../osiris_query/')
from src.query import load_frame

In [ ]:
df_vti = load_frame('betaalmail/s_sih_var_2019')
df_ooa = load_frame('betaalmail/s_ooa_aan_var_2019')
df_dos = load_frame('ooa/s_ooa_dos_var_BA1920_MATCHING_I')
df_opl = load_frame('referentie/r_opl')

In [ ]:
df_dos = df_dos.loc[df_dos['processtap'].str.contains('O_DATUM')]
df_dos = df_dos.loc[df_dos['gesloten_antwoord_code'] == 'JUNI']
df_dos = df_dos.loc[df_dos['faculteit'] == 'BETA']

In [ ]:
cols = ['studentnummer', 'opleiding', 'croho']
df_ooa_c = df_ooa.merge(df_vti[cols], on=['studentnummer', 'opleiding'])

In [ ]:
cols = ['ooa_id', 'gesloten_antwoord_code']
df_m = df_dos[cols].merge(df_ooa_c, on=['ooa_id'])

In [ ]:
df_m['opleiding'].value_counts()

In [ ]:
cols = ['studentnummer', 'croho', 'opleiding']
df_inlees = df_m[cols].copy()

In [ ]:
df_nast = df_inlees.loc[df_inlees['opleiding'] == 'NAST']['studentnummer']
df_wisk = df_inlees.loc[df_inlees['opleiding'] == 'WISK']['studentnummer']
df_schk = df_inlees.loc[df_inlees['opleiding'] == 'SCHK']['studentnummer']
df_inca = df_inlees.loc[df_inlees['opleiding'] == 'INCA']['studentnummer']
df_inku = df_inlees.loc[df_inlees['opleiding'] == 'INKU']['studentnummer']

In [ ]:
stud_nast = df_inlees['studentnummer'].isin(df_nast)
stud_wisk = df_inlees['studentnummer'].isin(df_wisk)
stud_schk = df_inlees['studentnummer'].isin(df_schk)
stud_inca = df_inlees['studentnummer'].isin(df_inca)
stud_inku = df_inlees['studentnummer'].isin(df_inku)

In [ ]:
# INKU/INCA
filt = stud_inca & stud_inku
df_inlees.loc[filt & (df_inlees['opleiding'] == 'INKU'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'INCA'
df_inlees.loc[filt & (df_inlees['opleiding'] == 'INCA'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'INKU'

# WISK/INCA
filt = stud_wisk & stud_inca
df_inlees.loc[filt & (df_inlees['opleiding'] == 'WISK'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'INCA'
df_inlees.loc[filt & (df_inlees['opleiding'] == 'INCA'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'WISK'

# NAST/SCHK
filt = stud_nast & stud_schk
df_inlees.loc[filt & (df_inlees['opleiding'] == 'SCHK'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'NAST'
df_inlees.loc[filt & (df_inlees['opleiding'] == 'NAST'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'SCHK'

# NAST/WISK
filt = stud_nast & stud_wisk
df_inlees.loc[filt & (df_inlees['opleiding'] == 'WISK'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'NAST'
df_inlees.loc[filt & (df_inlees['opleiding'] == 'NAST'), 'VOOR_ACTIVITEIT_BETA_TWIN'] = 'WISK'

In [ ]:
df_inlees.query("VOOR_ACTIVITEIT_BETA_TWIN == VOOR_ACTIVITEIT_BETA_TWIN").to_excel('output/inlees_bestand_beta.xlsx', index=False)